# NCEI
<br>

- 지점정보
    - 링크 내용은 동일함
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt(내용확인)
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv(파일다운로드)
- 국가 코드
    - Country list
        - 국가명과 국가 2글자 표현 짝 정리된 것
        - 후에 요청인자 중 stn 구성요소인 지점코드값의 국가 부분을 해석하는데에 사용
        - 293개국 존재
- 파일 리스트
    - File list
        - 파일목록
        - 파일명과 파일 크기가 나열됨
        - 후술될 요청인자 중 stn의 구성요소인 지점코드 값으로 활용할 수 있음
            - 국가 2글자, 망구분 1글자, 지점번호 9글자를 가져올 소스
        - 122236개 있음
- 실제 파일 리스트
    - data_climate/gsom-latest 내부에 존재하는 파일들
    - 122041개 있음

## 활용 library

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import urllib
from urllib.request import urlopen

## Country List와 File List 병합
- FileList의 파일명 5,6 번째 글자와 Country List의 FIPS ID가 동일하면 후자의 Country Name을 추가해준다

In [2]:
dfCountry = pd.read_csv('data_climate/country_list.csv', sep = ',',
                        encoding = 'euc-kr', engine = 'python')
dfFileList = pd.read_csv('data_climate/file_list.csv', sep = ',',
                  encoding = 'euc-kr', engine = 'python')

In [3]:
dfCountry

,FIPS ID,COUNTRY NAME
0,AA,ARUBA
1,AC,ANTIGUA AND BARBUDA
2,AF,AFGHANISTAN
3,AG,ALGERIA
4,AI,ASCENSION ISLAND
...,...,...
288,YY,"ST. MARTEEN, ST. EUSTATIUS, AND SABA"
289,ZA,ZAMBIA
290,ZI,ZIMBABWE
291,ZM,SAMOA


In [4]:
dfFileList

,FILE,SIZE
0,gsom_ACW00011604.csv,4036
1,gsom_ACW00011647.csv,34313
2,gsom_AE000041196.csv,130742
3,gsom_AEM00041194.csv,100242
4,gsom_AEM00041217.csv,85259
...,...,...
122231,gsom_ZI000067965.csv,114617
122232,gsom_ZI000067969.csv,116596
122233,gsom_ZI000067975.csv,200883
122234,gsom_ZI000067977.csv,84153


In [5]:
dfFileList['ID'] = dfFileList['FILE'].str[5:7]
dfFileList

,FILE,SIZE,ID
0,gsom_ACW00011604.csv,4036,AC
1,gsom_ACW00011647.csv,34313,AC
2,gsom_AE000041196.csv,130742,AE
3,gsom_AEM00041194.csv,100242,AE
4,gsom_AEM00041217.csv,85259,AE
...,...,...,...
122231,gsom_ZI000067965.csv,114617,ZI
122232,gsom_ZI000067969.csv,116596,ZI
122233,gsom_ZI000067975.csv,200883,ZI
122234,gsom_ZI000067977.csv,84153,ZI


In [6]:
dfCountryList = pd.merge(dfFileList, dfCountry,
                         how = 'left',
                         left_on = 'ID',
                         right_on = 'FIPS ID')
dfCountryList

,FILE,SIZE,ID,FIPS ID,COUNTRY NAME
0,gsom_ACW00011604.csv,4036,AC,AC,ANTIGUA AND BARBUDA
1,gsom_ACW00011647.csv,34313,AC,AC,ANTIGUA AND BARBUDA
2,gsom_AE000041196.csv,130742,AE,NaN,NaN
3,gsom_AEM00041194.csv,100242,AE,NaN,NaN
4,gsom_AEM00041217.csv,85259,AE,NaN,NaN
...,...,...,...,...,...
122231,gsom_ZI000067965.csv,114617,ZI,ZI,ZIMBABWE
122232,gsom_ZI000067969.csv,116596,ZI,ZI,ZIMBABWE
122233,gsom_ZI000067975.csv,200883,ZI,ZI,ZIMBABWE
122234,gsom_ZI000067977.csv,84153,ZI,ZI,ZIMBABWE


In [7]:
dfCountryList.isnull().sum()

FILE            0
SIZE            0
ID              0
FIPS ID         6
COUNTRY NAME    6
dtype: int64

In [8]:
dfCountryList[dfCountryList['FIPS ID'].isnull()]

,FILE,SIZE,ID,FIPS ID,COUNTRY NAME
2,gsom_AE000041196.csv,130742,AE,NaN,NaN
3,gsom_AEM00041194.csv,100242,AE,NaN,NaN
4,gsom_AEM00041217.csv,85259,AE,NaN,NaN
5,gsom_AEM00041218.csv,48310,AE,NaN,NaN
46733,gsom_NNM00078866.csv,54757,NN,NaN,NaN
52317,gsom_UCM00078988.csv,56684,UC,NaN,NaN


In [9]:
CountryNull = dfCountryList[dfCountryList['FIPS ID'].isnull()]['ID'].unique()
CountryNull

array(['AE', 'NN', 'UC'], dtype=object)

In [10]:
dfCountryList['FILENM'] = dfCountryList['FILE'].str[5:]
dfCountryList

,FILE,SIZE,ID,FIPS ID,COUNTRY NAME,FILENM
0,gsom_ACW00011604.csv,4036,AC,AC,ANTIGUA AND BARBUDA,ACW00011604.csv
1,gsom_ACW00011647.csv,34313,AC,AC,ANTIGUA AND BARBUDA,ACW00011647.csv
2,gsom_AE000041196.csv,130742,AE,NaN,NaN,AE000041196.csv
3,gsom_AEM00041194.csv,100242,AE,NaN,NaN,AEM00041194.csv
4,gsom_AEM00041217.csv,85259,AE,NaN,NaN,AEM00041217.csv
...,...,...,...,...,...,...
122231,gsom_ZI000067965.csv,114617,ZI,ZI,ZIMBABWE,ZI000067965.csv
122232,gsom_ZI000067969.csv,116596,ZI,ZI,ZIMBABWE,ZI000067969.csv
122233,gsom_ZI000067975.csv,200883,ZI,ZI,ZIMBABWE,ZI000067975.csv
122234,gsom_ZI000067977.csv,84153,ZI,ZI,ZIMBABWE,ZI000067977.csv


## NCEI 파일 목록을 가져오기

In [11]:
path = 'data_climate/gsom-latest'
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]

In [12]:
len(file_list_csv)

122041

가지고 있는 파일과 파일리스트를 merge하여 파일 보유 정보를 확인하기

In [13]:
dfRealFileList = pd.DataFrame(file_list_csv, columns = ['RealFileNM'])

In [14]:
dfRealFileList

,RealFileNM
0,ACW00011604.csv
1,ACW00011647.csv
2,AE000041196.csv
3,AEM00041194.csv
4,AEM00041217.csv
...,...
122036,ZI000067969.csv
122037,ZI000067975.csv
122038,ZI000067977.csv
122039,ZI000067983.csv


In [15]:
dfFinalFileList = pd.merge(dfRealFileList, dfCountryList,
                           how = 'left',
                           left_on = 'RealFileNM',
                           right_on = 'FILENM')
dfFinalFileList

,RealFileNM,FILE,SIZE,ID,FIPS ID,COUNTRY NAME,FILENM
0,ACW00011604.csv,gsom_ACW00011604.csv,4036.0,AC,AC,ANTIGUA AND BARBUDA,ACW00011604.csv
1,ACW00011647.csv,gsom_ACW00011647.csv,34313.0,AC,AC,ANTIGUA AND BARBUDA,ACW00011647.csv
2,AE000041196.csv,gsom_AE000041196.csv,130742.0,AE,NaN,NaN,AE000041196.csv
3,AEM00041194.csv,gsom_AEM00041194.csv,100242.0,AE,NaN,NaN,AEM00041194.csv
4,AEM00041217.csv,gsom_AEM00041217.csv,85259.0,AE,NaN,NaN,AEM00041217.csv
...,...,...,...,...,...,...,...
122036,ZI000067969.csv,gsom_ZI000067969.csv,116596.0,ZI,ZI,ZIMBABWE,ZI000067969.csv
122037,ZI000067975.csv,gsom_ZI000067975.csv,200883.0,ZI,ZI,ZIMBABWE,ZI000067975.csv
122038,ZI000067977.csv,gsom_ZI000067977.csv,84153.0,ZI,ZI,ZIMBABWE,ZI000067977.csv
122039,ZI000067983.csv,gsom_ZI000067983.csv,193672.0,ZI,ZI,ZIMBABWE,ZI000067983.csv


In [16]:
dfFinalFileList.isnull().sum()

RealFileNM       0
FILE            10
SIZE            10
ID              10
FIPS ID         16
COUNTRY NAME    16
FILENM          10
dtype: int64

In [18]:
# 파일리스트를 참고해도 파악 안되는 곳
# 필요시 수기로 찾아봐야 함
dfFinalFileList[dfFinalFileList['FIPS ID'].isnull()]

,RealFileNM,FILE,SIZE,ID,FIPS ID,COUNTRY NAME,FILENM
2,AE000041196.csv,gsom_AE000041196.csv,130742.0,AE,NaN,NaN,AE000041196.csv
3,AEM00041194.csv,gsom_AEM00041194.csv,100242.0,AE,NaN,NaN,AEM00041194.csv
4,AEM00041217.csv,gsom_AEM00041217.csv,85259.0,AE,NaN,NaN,AEM00041217.csv
5,AEM00041218.csv,gsom_AEM00041218.csv,48310.0,AE,NaN,NaN,AEM00041218.csv
46730,NNM00078866.csv,gsom_NNM00078866.csv,54757.0,NN,NaN,NaN,NNM00078866.csv
52313,UCM00078988.csv,gsom_UCM00078988.csv,56684.0,UC,NaN,NaN,UCM00078988.csv
56124,US1AZYV0208.csv,NaN,NaN,NaN,NaN,NaN,NaN
59946,US1COLR1064.csv,NaN,NaN,NaN,NaN,NaN,NaN
72280,US1MNHN0366.csv,NaN,NaN,NaN,NaN,NaN,NaN
74840,US1NCBC0173.csv,NaN,NaN,NaN,NaN,NaN,NaN
